In [1]:
import os
import shutil
import random
import numpy as np
import pandas as pd

In [2]:
seed = 42
np.random.seed(seed)

In [18]:
directory = 'Datasets-Folders/ISIC_2019_Training_Input/'

train = 'Datasets/train/'
valid = 'Datasets/valid/'
test = 'Datasets/test/'
try:
    os.makedirs(train +'benign/')
    os.makedirs(train +'malignant/')
    os.makedirs(test +'benign/')
    os.makedirs(test +'malignant/')
    os.makedirs(valid +'benign/')
    os.makedirs(valid +'malignant/')
except:
    print("Files are already created.")

Files are already created.


In [8]:
test_examples = train_examples = validation_examples = 0

import csv

with open("Datasets-Folders/ISIC_2019_Training_Input/Labels.csv", "r") as f:
    next(f)
    for line in f:
        line_split = line.split(',')
        image = line_split[0]
        benign_malignant = line_split[1]
        
        rn = random.random()
        # 0.8, 0.1, 0.1
        if rn < 0.8:
            location = train
            train_examples+=1
        elif rn > 0.8 and rn < 0.9:
            location = valid
            validation_examples+=1
        else:
            location = test
            test_examples+=1
            
        if int(float(benign_malignant))==0:
            shutil.copy(src=directory+image+'.jpg', 
                        dst=location+'benign/'+image+'.jpg')
        else:
            shutil.copy(src=directory+image+'.jpg', 
                        dst=location+'malignant/'+image+'.jpg')
            
print(train_examples, validation_examples, test_examples)

20295 2565 2471


In [9]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [10]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [11]:
height, width = 175, 175
batch_size = 256
num_classes = 1

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

base_model = tf.keras.applications.xception.Xception(include_top=False)
                                                     
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [13]:
for layer in base_model.layers:
    layer.trainable = False

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=1/255,
    data_format='channels_last',
    dtype=tf.float32)

train_generator = train_generator.flow_from_directory(
    directory=train,
    target_size=(height, width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
    seed=42
)

valid_generator = ImageDataGenerator(rescale=1/255,
                                     dtype=tf.float32)


valid_generator = valid_generator.flow_from_directory(
    directory=valid,
    target_size=(height, width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
    seed=42
)

test_generator = ImageDataGenerator(rescale=1/255,
                                     dtype=tf.float32)


test_generator = test_generator.flow_from_directory(
    directory=test,
    target_size=(height, width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
    seed=42
)

Found 24305 images belonging to 2 classes.
Found 4795 images belonging to 2 classes.
Found 4718 images belonging to 2 classes.


In [15]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
metrics = [
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc')
]
model.compile(optimizer='adam', 
              loss=loss,
              metrics=metrics)

In [16]:
 model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [19]:
import warnings
warnings.filterwarnings('ignore')

tf.keras.backend.clear_session()

In [20]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=1
)

94/94 [==============================] - 727s 8s/step - loss: 0.4774 - accuracy: 0.8182 - precision: 0.4629 - recall: 0.1222 - auc: 0.6755 - val_loss: 0.3974 - val_accuracy: 0.8411 - val_precision: 0.7394 - val_recall: 0.1504 - val_auc: 0.7512


In [21]:
model.evaluate_generator(valid_generator,
                             steps=STEP_SIZE_VALID,
                             verbose=1)

18/18 [==============================] - 76s 4s/step - loss: 0.3930 - accuracy: 0.8427 - precision: 0.7160 - recall: 0.1459 - auc: 0.7511


[0.3930394649505615,
 0.8426649570465088,
 0.7160493731498718,
 0.14591194689273834,
 0.7510774731636047]

In [22]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
model.evaluate_generator(test_generator,
                             steps=STEP_SIZE_TEST,
                             verbose=1)

18/18 [==============================] - 94s 5s/step - loss: 0.4020 - accuracy: 0.8424 - precision: 0.8294 - recall: 0.1683 - auc: 0.7521


[0.4020390510559082,
 0.8424479365348816,
 0.8294117450714111,
 0.16825775802135468,
 0.7520976662635803]